In [1]:
from sklearn.model_selection import KFold
import pandas as pd

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: konggedzu (imucs). Use `wandb login --relogin` to force relogin


True

In [3]:
sweep_config = {
    'method': 'random'
}

In [4]:
metric = {
    'name': 'Grand Mean',
    'goal': 'maximize'
}

sweep_config['metric'] = metric

In [5]:
parameters_dict = {
    'dropout': {
          'values': [0.4]
        },

    'learning_rate': {
        'values': [0.0005]
        },

    'batch_size': {
        'values': [4096]
        }
}

In [6]:
sweep_config['parameters'] = parameters_dict

In [7]:
sweep_id = wandb.sweep(sweep_config, project="sub_loc_train_no_aug")

Create sweep with ID: z4lycq99
Sweep URL: https://wandb.ai/imucs/sub_loc_train_no_aug/sweeps/z4lycq99


In [8]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'Grand Mean'},
 'parameters': {'batch_size': {'values': [4096]},
                'dropout': {'values': [0.4]},
                'learning_rate': {'values': [0.0005]}}}


In [9]:
feature_pd = pd.read_csv('/home/kongge/projects/new_protT5/data/DPC_T5_578_right.csv')
labels_pd = pd.read_csv("/home/kongge/projects/new_protT5/data/mutil_label_578.csv")

In [10]:
from dataAug.tools import MLDA

In [11]:
from dataAug.all_tools import dataAugSMOTE

In [12]:

smote_multiple = {}

In [13]:
import time
from classify.targeTools import testThresholdFive, Accuracy, countScore
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
import torch
from classify.Classify_adjust import ModelClassify
def train_and_val(config=None):
    with wandb.init(config=config):
        config = wandb.config
        kf = KFold(n_splits=10, shuffle=True)
        model_discord = []
        for train_index, test_index in kf.split(feature_pd):
            train_true_data = feature_pd.iloc[train_index]
            train_true_label = labels_pd.iloc[train_index]

            multi_label_samples = train_true_label[(train_true_label.sum(axis=1) >= 2)]
            multi_label_indices = multi_label_samples.index
            multi_features_samples = train_true_data.loc[multi_label_indices]
            ML_G_X, ML_G_y = MLDA(multi_features_samples, multi_label_samples, 5)

            feature_all = pd.concat([train_true_data, ML_G_X], axis=0)
            label_all = pd.concat([train_true_label, ML_G_y], axis=0)

            test_data = feature_pd.iloc[test_index]
            test_label = labels_pd.iloc[test_index]

            datasetTrain = TensorDataset(torch.tensor(feature_all.values), torch.tensor(label_all.values))
            batch_size = config.batch_size
            dataloaderTrain = DataLoader(datasetTrain, batch_size=batch_size, shuffle=True)

            datasetTest = TensorDataset(torch.tensor(test_data.values), torch.tensor(test_label.values))

            batch_size = len(datasetTest)
            dataloaderTest = DataLoader(datasetTest, batch_size=batch_size, shuffle=False)

            model = ModelClassify(drop_rate=config.dropout, num_class=5, feature_num=1424)
            criterion = torch.nn.BCELoss()
            optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
            import warnings
            warnings.filterwarnings("ignore")
            GMList = {}
            epochs = 150
            for epoch in range(epochs):
                model.train()
                total_loss = 0.0
                for idx, data in enumerate(dataloaderTrain, 0):
                    inputs, labels = data
                    labels = labels.float()
                    inputs = inputs.float()
                    out = model(inputs)
                    loss = criterion(out, labels)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item()
                avg_loss = total_loss / len(dataloaderTrain)
                threshold = 0.5
                labels_cov = torch.where(out > threshold, torch.tensor(1), torch.tensor(0))
                print(f"Epoch [{epoch+1}/{epochs}], Average Loss: {avg_loss:.4f}, ACC: {Accuracy(labels.int(), labels_cov)}")
            GMScore1 = testThresholdFive(epoch, model, dataloaderTest, class_num=5)
            GMList[epoch] = GMScore1
            t = time.time()
            best_key = max(GMList, key=GMList.get)
            best_value = GMList[best_key]

            best_value = [x.item() if isinstance(x, torch.Tensor) else x for x in best_value]
            model_discord.append(best_value)

            # model.eval()
            # with torch.no_grad():
            #     for idxTest, dataTest in enumerate(dataloaderTest, 0):
            #         inputsTest, targetsFive = dataTest
            #         inputsTest = inputsTest.float()
            #         test_output = model(inputsTest)
            # test_labels_cov = torch.where(test_output > threshold, torch.tensor(1), torch.tensor(0))
            # GMScore1 = countScore(targetsFive.int(), test_labels_cov)
            # GMList[epoch] = GMScore1
            # t = time.time()
            # best_key = max(GMList, key=GMList.get)
            # best_value = GMList[best_key]
            # best_value = [x.item() if isinstance(x, torch.Tensor) else x for x in best_value]
            # model_discord.append(best_value)
        model_discord_column_means = [sum(col) / len(col) for col in zip(*model_discord)]
        smote_multiple[int(t)] = model_discord_column_means
        wandb.log({"Grand Mean": model_discord_column_means[0]})

In [14]:
wandb.agent(sweep_id, train_and_val, count=3)

wandb: Agent Starting Run: kvcxalcx with config:
wandb: 	batch_size: 4096
wandb: 	dropout: 0.4
wandb: 	learning_rate: 0.0005


Epoch [1/150], Average Loss: 0.7288, ACC: 0.22166666666666648
Epoch [2/150], Average Loss: 0.6854, ACC: 0.2740163934426231
Epoch [3/150], Average Loss: 0.6520, ACC: 0.359699453551913
Epoch [4/150], Average Loss: 0.6220, ACC: 0.4006830601092901
Epoch [5/150], Average Loss: 0.5991, ACC: 0.4193715846994542
Epoch [6/150], Average Loss: 0.5756, ACC: 0.4578142076502736
Epoch [7/150], Average Loss: 0.5606, ACC: 0.48491803278688533
Epoch [8/150], Average Loss: 0.5383, ACC: 0.5020765027322408
Epoch [9/150], Average Loss: 0.5193, ACC: 0.5296174863387978
Epoch [10/150], Average Loss: 0.5002, ACC: 0.5495628415300545
Epoch [11/150], Average Loss: 0.4791, ACC: 0.5581967213114751
Epoch [12/150], Average Loss: 0.4633, ACC: 0.5741256830601092
Epoch [13/150], Average Loss: 0.4488, ACC: 0.5879781420765023
Epoch [14/150], Average Loss: 0.4396, ACC: 0.605901639344262
Epoch [15/150], Average Loss: 0.4224, ACC: 0.6257923497267756
Epoch [16/150], Average Loss: 0.4126, ACC: 0.6443169398907097
Epoch [17/150], A

Grand Mean,▁
Grand Mean,0.80179


wandb: Agent Starting Run: 98w2f5dv with config:
wandb: 	batch_size: 4096
wandb: 	dropout: 0.4
wandb: 	learning_rate: 0.0005


Epoch [1/150], Average Loss: 0.7375, ACC: 0.19783999999999993
Epoch [2/150], Average Loss: 0.6931, ACC: 0.29136
Epoch [3/150], Average Loss: 0.6515, ACC: 0.34325333333333374
Epoch [4/150], Average Loss: 0.6309, ACC: 0.3998400000000006
Epoch [5/150], Average Loss: 0.6039, ACC: 0.4218933333333342
Epoch [6/150], Average Loss: 0.5807, ACC: 0.45690666666666685
Epoch [7/150], Average Loss: 0.5596, ACC: 0.48573333333333313
Epoch [8/150], Average Loss: 0.5409, ACC: 0.4976533333333333
Epoch [9/150], Average Loss: 0.5237, ACC: 0.4985599999999995
Epoch [10/150], Average Loss: 0.5034, ACC: 0.5393066666666664
Epoch [11/150], Average Loss: 0.4817, ACC: 0.5751733333333334
Epoch [12/150], Average Loss: 0.4654, ACC: 0.5699199999999993
Epoch [13/150], Average Loss: 0.4557, ACC: 0.598773333333333
Epoch [14/150], Average Loss: 0.4383, ACC: 0.6162666666666663
Epoch [15/150], Average Loss: 0.4196, ACC: 0.6447999999999999
Epoch [16/150], Average Loss: 0.4071, ACC: 0.6342666666666669
Epoch [17/150], Average L

wandb: Ctrl + C detected. Stopping sweep.


In [15]:
smote_multiple

{1695280083: [0.8017896890640259,
  0.7179975801572899,
  0.7964508973583383,
  0.8045120000839233,
  0.8671203970909118,
  0.8228675484657287],
 1695280261: [0.8526856303215027,
  0.7995160314579552,
  0.849067352288768,
  0.8600121021270752,
  0.8888233482837677,
  0.8660093307495117]}

In [16]:
import json
file = open("/home/kongge/projects/new_protT5/data/dictionary_data_no_aug.json", "w")
json.dump(smote_multiple, file)
file.close()